In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [ ]:
browser = webdriver.Chrome('/Users/sky/Downloads/chromedriver')
url = 'https://www.saramin.co.kr/zf_user/jobs/hot100'
browser.get(url)
browser.find_element_by_xpath('//*[@id="search_panel_wrapper"]/form/fieldset/div/div[1]/div/div/button[7]').click()
time.sleep(1)
soup = BeautifulSoup(browser.page_source)

In [ ]:
rank_rookie = []

for tag in soup.select('#content > div.recruit_hot_wrap > div.recruit_hot_list > div.careers > div > ul > li'):
    
    company = {}    
    company['company'] = tag.select('div.area_rank > div.rank_company_info > a > span')[0].text
    company['title'] = tag.select('div.area_detail > a.tit > span')[0].text
    company['career'] = tag.select('div.area_detail > div > span')[0].text
    company['degree'] = tag.select('div.area_detail > div > span')[1].text
    company['work_type'] = tag.select('div.area_detail > div > span')[2].text
    
    # 근무지 없는 경우 발생 -> 예외 처리
    try:
        company['work_place'] = tag.select('div.area_detail > div > span')[3].text
    except:
        company['work_place'] = ""

    company['duty'] = tag.select('.area_detail > ul')[0].text
    company['due_date'] = tag.select('div.area_detail > dl > dd')[0].text
    
    rank_rookie.append(company)
    
rank_rookie

In [ ]:
df = pd.DataFrame(rank_rookie)
# regex=True가 있어야 정규표현식이 작동함
df = df.replace('\n', '', regex=True)
# 추천하는 열은 지우기 위해서
df = df.drop([0])
# csv로 저장
df.to_csv('rank_rookie_saramin.csv', encoding='utf8')

데이터가 적어서 바로 DB에 넣는 코드도 추가

In [ ]:
import pymysql
import pandas as pd

In [ ]:
# CSV 읽기
rookie_s = pd.read_csv('/Users/sky/class_python/1.Crawling/rank_rookie_saramin.csv', index_col=0, encoding='utf8')
rookie_s.head(3)

In [ ]:
# DB연동
conn = pymysql.connect(host='localhost',
                       database='test', # 자기 데이터 베이스에 맞게 접속
                       user='root',
                       passwd='mlb10200', #자기 비밀번호
                       charset='utf8',
                       local_infile=1) #혹시나 안되면 이거 켜서 접속

cursor = conn.cursor()

In [ ]:
# test용 TABLE 생성
sql = '''
    CREATE TABLE rookie_s(
        company varchar(255),
        title varchar(255),
        career varchar(255),
        degree varchar(255),
        work_type varchar(255),
        work_place varchar(255),
        duty varchar(255),
        due_date varchar(255)
    )
      '''
cursor.execute(sql)
#커밋은 필수
conn.commit()

In [ ]:
# DB INSERT

value_list = []
for index, row in rookie_s.iterrows():
    tu = ( row.company, row.title, row.duty, row.career,
           row.degree, row.work_place, row.work_type, row.due_date ) 
    value_list.append(tu)
    
cursor.executemany('''
                  INSERT INTO rookie_s (company, title, duty, career, degree, work_place, work_type, due_date)
                  values ( %s, %s, %s, %s, %s, %s, %s, %s )
                  ''', value_list)

conn.commit()
#conn.close()

In [ ]:
# 테이블 조회

sql = 'SELECT * FROM test.rookie_s'
cursor.execute(sql)
result = cursor.fetchall()
print(result)
#conn.close()

In [ ]:
# TABLE 삭제
# sql_drop = '''
#             DROP TABLE rookie_s
# '''
# cursor.execute(sql_drop)
# conn.commit()